In [53]:
from requests import Session
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

from datetime import datetime

HEADERS = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) '\
                         'AppleWebKit/537.36 (KHTML, like Gecko) '\
                         'Chrome/75.0.3770.80 Safari/537.36'}

In [65]:
import sys
# to import local fuctions
sys.path.insert(0, '../tools')
filepath = '/Users/renovieira/Desktop/swing_trading/output/trades.feather'
# Read file
trades = pd.read_feather(filepath)

In [74]:
trades['earnings_difference'].astype(str) + trades['symbol']

0          nanGCAP
1          nanTLRD
2          nanTTPH
3           nanDNR
4          nanASNA
            ...   
4993726    nanMARA
4993727     nanCBL
4993728     nanCCL
4993729     nanAAL
4993730     nanSPY
Length: 4993731, dtype: object

In [62]:
"""
Search for symbol's link in Simply Wall Street
"""
# Create Session
s = Session()

# Add headers
s.headers.update(HEADERS)
symbol = 'pbr'

# JSON Key Field
url = f'https://autocomplete.tipranks.com/api/Autocomplete/getAutoCompleteNoSecondary/?name={symbol}'

# Request and transform response in json
search = s.get(url)
json = search.json()

# If there's data
if len(json) > 0:

    # Extract symbol and update url
    final_symbol = json[0]['value']
    url = f'https://www.tipranks.com/api/stocks/getData/?name={final_symbol}'

    # Extract data
    data = s.get(url)
    json = data.json()

In [63]:
json['ptConsensus']

[{'period': 3,
  'bench': 1,
  'priceTarget': None,
  'priceTargetCurrency': None,
  'priceTargetCurrencyCode': None,
  'high': None,
  'low': None},
 {'period': 0,
  'bench': 0,
  'priceTarget': 10.87,
  'priceTargetCurrency': 1,
  'priceTargetCurrencyCode': 'USD',
  'high': 13.0,
  'low': 7.5}]

In [64]:
json['portfolioHoldingData']['bestPriceTarget']

37.0
1.0
0.0
3724.09


In [77]:
import requests
import pandas as pd
from datetime import datetime

import sys
# to import local fuctions
sys.path.insert(0, '../../tools')

from keys.discord_keys import client_id
from extract.fundamentals import symbol_data, search_symbol
from extract.tipranks import analyst_recommendation

def send_message_to_discord(signal
                            , webhook=client_id.webhook):
    # Define path to file
    filepath = '/Users/renovieira/Desktop/swing_trading/output/trades.feather'

    # Read file
    trades = pd.read_feather(filepath)

    # Figure out today's date and keep only today's buys
    today = datetime.today().strftime("%Y-%m-%d")

    # Find today's buys
    buy_me = trades[(trades[signal] == True) & (trades['just_date'].astype(str) == today)]

    # Extract Price from Valuation
    results = []
    for symbol in buy_me['symbol']:

        recommendation = analyst_recommendation(symbol)
        results.append(recommendation)

    # Merge results
    analysts = pd.DataFrame(results, columns = ['symbol', 'consensus', 'buys', 'hold', 'sells', 'target', 'sentiment'])
    buy_me = pd.merge(buy_me, analysts, on =['symbol'], how='left')

    # Potential
    buy_me['potential'] = (((buy_me['target'] / buy_me['close_price_x']) - 1) * 100).round()

    # Create message
    buy_me = buy_me.fillna('nan')
    buy_me['earnings_difference'] = buy_me['earnings_difference'].astype(str)
    
    return buy_me

In [79]:
buy_me = send_message_to_discord('buy_signal?7')

In [81]:
buy_me

,just_date,symbol,close_price_x,buy_signal?1,sell_signal?1,buy_signal?2,sell_signal?2,buy_signal?3,sell_signal?3,buy_signal?4,...,eps_surprise,earnings_difference,previous_day_price,consensus,buys,hold,sells,target,sentiment,potential
0,2020-08-10,GBCI,36.840000,nan,nan,nan,nan,nan,nan,nan,...,nan,18.0,nan,nan,0.0,0.0,0.0,38.00,NAN,3.0
1,2020-08-10,MRCY,79.190002,True,nan,nan,nan,nan,nan,nan,...,nan,6.0,nan,buy,1.0,0.0,0.0,94.33,1,19.0
2,2020-08-10,HR,28.950001,nan,nan,nan,nan,nan,nan,nan,...,nan,5.0,nan,nan,0.0,0.0,0.0,34.00,0.5,17.0
3,2020-08-10,SLM,7.190000,nan,nan,nan,nan,nan,nan,nan,...,nan,19.0,nan,buy,1.0,2.0,0.0,9.25,0.5,29.0


In [89]:
buy_me['earnings_difference'] + buy_me['symbol'] + buy_me['sentiment'].astype(str)

0    18.0GBCINAN
1     6.0MRCY1.0
2       5.0HR0.5
3     19.0SLM0.5
dtype: object

In [85]:

buy_me['string'] = (buy_me['symbol'].replace(' ', '') + ' @' + buy_me['close_price_x'].round(2).astype(str) +
                    ' // ' + buy_me['consensus'].str.upper() + ' Target: ' + buy_me['target'].astype(str) + ' (' +
                    buy_me['potential'].astype(str).str[:-2] + '%)' + ' // Bullish Sentiment: ' + buy_me['sentiment'] +
                    str(buy_me['earnings_difference']) + ' days since earnings.')


TypeError: can only concatenate str (not "float") to str